# Ingest Careers for fact_employability

### Config

In [1]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [2]:
remote_path = 'http://pacha.datawheel.us/datachile/education/empleabilidad/'
local_path = '../data/'

### Imports

In [3]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

In [4]:
df = download_file(remote_path+'ids/',local_path,'carreras_verificacion_manual.csv')
df1 = pd.DataFrame(df.FOS.str.split(' - ',1).tolist(),columns = ['career_group_id','career_group'])
df1['career_group_en'] = df1['career_group']
df = pd.concat([df, df1], axis=1)
df = df.drop('FOS', 1)
df = df.astype({'career_id':'int','career_group_id':'int'})
df['career_group'] = df.career_group.str.strip()
df['career_group_en'] = df.career_group_en.str.strip()
df['career_group'] = df['career_group'].map({'Humanities': 'Humanidades', 'Social Sciences': 'Ciencias Sociales','Engineering and Technology':'Ingeniería y Tecnología','Medical and Health Sciences':'Ciencias Médicas y de la Salud','Agricultural Sciences':'Ciencias de la Agricultura','Natural Sciences':'Ciencias Naturales'})
df

Downloading... http://pacha.datawheel.us/datachile/education/empleabilidad/ids/carreras_verificacion_manual.csv


,career,career_en,career_id,career_group_id,career_group,career_group_en
0,Actuación,Actuación,1,6,Humanidades,Humanities
1,Administración de Artes Culinarias y Servicios,Administración de Artes Culinarias y Servicios,2,6,Humanidades,Humanities
2,Administración de Empresas,Administración de Empresas,3,5,Ciencias Sociales,Social Sciences
3,Administración de Empresas de Turismo,Administración de Empresas de Turismo,4,5,Ciencias Sociales,Social Sciences
4,Administración de Hoteles y Restaurantes,Administración de Hoteles y Restaurantes,5,5,Ciencias Sociales,Social Sciences
5,Administración de Negocios Internacionales,Administración de Negocios Internacionales,6,5,Ciencias Sociales,Social Sciences
6,Administración de Negocios y Ventas,Administración de Negocios y Ventas,7,5,Ciencias Sociales,Social Sciences
7,Administración de Redes y Soporte Computacional,Administración de Redes y Soporte Computacional,8,2,Ingeniería y Tecnología,Engineering and Technology
8,Administración de Servicios,Administración de Servicios,9,5,Ciencias Sociales,Social Sciences
9,Administración Gastronómica Internacional,Administración Gastronómica Internacional,10,5,Ciencias Sociales,Social Sciences


### Ingest

In [5]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'education', 'dim_careers')

DROP TABLE IF EXISTS education.dim_careers;
CREATE TABLE "education"."dim_careers" (
"career" TEXT,
  "career_en" TEXT,
  "career_id" INTEGER,
  "career_group_id" INTEGER,
  "career_group" TEXT,
  "career_group_en" TEXT
)
COPY "education"."dim_careers" ("career","career_en","career_id","career_group_id","career_group","career_group_en") FROM STDIN WITH CSV HEADER DELIMITER ',';
